<h1 align="center">Colombia Covid-19 Bar Chart Race by City in Tableau</h1>

The goal of this project is to create a Bar Chart Race with the Covid-19 cases by cities in Colombia. The data source will be taken from the goverment oficial repository [Datos abiertos](https://www.datos.gov.co/Salud-y-Protecci-n-Social/Casos-positivos-de-COVID-19-en-Colombia/gt2j-8ykr/data) specifically from their .csv file [.csv file](https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?accessType=DOWNLOAD). We will process this data to get a dataframe that we will export to a .hyper file which is a Tableau extension that works well with big tables

## 1. Importing and read the data
<p>Importing the libraries needed for read the data source and create the dataframe</p>

In [16]:
# Importing libraries to be able to work with the data

import pandas as pd
import numpy as np

In [17]:
# Reading the data from the official data source. This process will take a while because this data is updated daily and the file size will be growing day by day. Just be patient, pandas will process all the data. 

raw_df = pd.read_csv('https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?accessType=DOWNLOAD')

# How many columns and rows has this dataframe?. Let's check

raw_df.shape

(902747, 22)

In [18]:
# The raw dataframe information and structure

raw_df.head()

,ID de caso,Fecha de notificación,Código DIVIPOLA,Ciudad de ubicación,Departamento o Distrito,atención,Edad,Sexo,Tipo,Estado,...,Fecha de muerte,Fecha diagnostico,Fecha recuperado,fecha reporte web,Tipo recuperación,Codigo departamento,Codigo pais,Pertenencia etnica,Nombre grupo etnico,ubicación recuperado
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,En estudio,Leve,...,NaN,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR,11,NaN,Otro,NaN,Casa
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,...,NaN,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR,76,724.0,Negro,NaN,Casa
2,3,2020-03-07T00:00:00.000,5001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,...,NaN,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,PCR,5,724.0,Otro,NaN,Casa
3,4,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,...,NaN,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,Casa
4,5,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,...,NaN,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,Casa


## 2. Cleaning the data
<p>Clean and filter the dataframe which will allows us working with the specific data needed for our project</p>

In [19]:
# Checking all the column names to determine which of them we will use for our analysis

raw_df.dtypes

ID de caso                    int64
Fecha de notificación        object
Código DIVIPOLA               int64
Ciudad de ubicación          object
Departamento o Distrito      object
atención                     object
Edad                          int64
Sexo                         object
Tipo                         object
Estado                       object
País de procedencia          object
FIS                          object
Fecha de muerte              object
Fecha diagnostico            object
Fecha recuperado             object
fecha reporte web            object
Tipo recuperación            object
Codigo departamento           int64
Codigo pais                 float64
Pertenencia etnica           object
Nombre grupo etnico          object
ubicación recuperado         object
dtype: object

In [20]:
# Select the columns that we will need to create our dataframe. In this case I only need three columns 'ID de caso', 'Fecha de notificación' and 'Ciudad de ubicación'

df_short = raw_df[['ID de caso', 'Fecha de notificación', 'Ciudad de ubicación']]

# See that the 'Fecha de notificación' data type changed. We need to change that in the data manipulation step.

df_short.head()

,ID de caso,Fecha de notificación,Ciudad de ubicación
0,1,2020-03-02T00:00:00.000,Bogotá D.C.
1,2,2020-03-06T00:00:00.000,Guadalajara de Buga
2,3,2020-03-07T00:00:00.000,Medellín
3,4,2020-03-09T00:00:00.000,Medellín
4,5,2020-03-09T00:00:00.000,Medellín


## 3. Manipulating the data
<p>Manipulate the data to obtain the dataset with the information that will be necessary in the Data Visualization step</p>

In [21]:
# Rename the columns names and normalizing them with upper case letters

df_short.rename(columns={'ID de caso': 'ID', 'Fecha de notificación': 'FECHA_CONTAGIO', 'Ciudad de ubicación': 'CIUDAD'}, inplace=True)
df_short.dtypes

ID                 int64
FECHA_CONTAGIO    object
CIUDAD            object
dtype: object

In [22]:
# Setting the datetime dtype

df_short['FECHA_CONTAGIO'] = pd.to_datetime(df_short['FECHA_CONTAGIO']) 
df_short.dtypes

ID                         int64
FECHA_CONTAGIO    datetime64[ns]
CIUDAD                    object
dtype: object

In [23]:
# Gruping the dataframe to aggregate a new column with the daily cases by city

df_casos = df_short.groupby(['ID', 'FECHA_CONTAGIO', 'CIUDAD']).CIUDAD.agg('count').to_frame('CASOS_DIARIOS').reset_index()
df_casos.head(10)

,ID,FECHA_CONTAGIO,CIUDAD,CASOS_DIARIOS
0,1,2020-03-02,Bogotá D.C.,1
1,2,2020-03-06,Guadalajara de Buga,1
2,3,2020-03-07,Medellín,1
3,4,2020-03-09,Medellín,1
4,5,2020-03-09,Medellín,1
5,6,2020-03-10,Itagüí,1
6,7,2020-03-08,Cartagena de Indias,1
7,8,2020-03-09,Bogotá D.C.,1
8,9,2020-03-08,Bogotá D.C.,1
9,10,2020-03-12,Bogotá D.C.,1


In [24]:
# Again we will group the dataframe by city and aggregate a new column with the acummulated sum of cases. This is fundamental to create the bar chart race in Tableu

df_casos['CASOS_TOTALES'] = df_casos.groupby(['CIUDAD']).CASOS_DIARIOS.agg('cumsum')
df_casos.head()

,ID,FECHA_CONTAGIO,CIUDAD,CASOS_DIARIOS,CASOS_TOTALES
0,1,2020-03-02,Bogotá D.C.,1,1
1,2,2020-03-06,Guadalajara de Buga,1,1
2,3,2020-03-07,Medellín,1,1
3,4,2020-03-09,Medellín,1,2
4,5,2020-03-09,Medellín,1,3


In [25]:
# Checkin the number of rows and columns in our dataframe

df_casos.shape

(902747, 5)

In [26]:
# Looking if the dataframe has any null/nan values

df_casos.isnull().values.any()

False

In [27]:
# Check our dataframe dtypes

df_casos.dtypes

ID                         int64
FECHA_CONTAGIO    datetime64[ns]
CIUDAD                    object
CASOS_DIARIOS              int64
CASOS_TOTALES              int64
dtype: object

## 4. Export the data
<p>We will work the visualization with Tableu so, we need to export our pandas dataframe into a file that can be recognized by the Tableu software. We need to download the following tools to be able to export and connect with Tableau:</p>

[Pandleau](https://pypi.org/project/pandleau/)

[TableauSDK](https://help.tableau.com/current/api/sdk/en-us/help.htm)

In [28]:
# We need to install the following library to be able to import our pandas dataframe into a .hyper file that will works in Tableau. 

from pandleau import *
from tableausdk import *

You are using the Tableau SDK, please save the output as .tde format
You are using the Extract API 2.0, please save the output as .hyper format


In [30]:
# Converting our dataframe into a Tableau object

df_colombia_covid19 = pandleau(df_casos)

# Creating a .hyper extract
# The .hyper is the current file extension supported in the last version of Tableau Personal, the .tde extension files are not currently supported by the Pandleau library

df_colombia_covid19.to_tableau('df_colombia_covid19.hyper', add_index=False)
print('the file df_colombia_covid19.hyper as been created')

Table 'Extract' does not exist in extract df_colombia_covid19.hyper, creating.
processing table: 902747it [00:48, 18678.03it/s]
the file df_colombia_covid19.hyper as been created
